In [61]:
# Current directory
import os
os.chdir('E:\Experiment\work')

### 导入数据


In [62]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_val_predict, train_test_split

X_new=pd.read_csv(r'train_feature/feature/ESM Cambrian.csv',header=None)
y_new= pd.read_csv(r'E:/Experiment\work\data/label_train.csv',header=None)

X_new1=pd.read_csv(r"teat_feature\DDE+DPC+CSKAAP+AAC+ESMC300.csv",header=None)
y_new1= pd.read_csv(r'E:/Experiment\work\data/label_test.csv',header=None)


print(X_new.shape)
print(y_new.shape)
print(X_new1.shape)
print(y_new1.shape)

X_new = np.array(X_new)
y_new = np.array(y_new).ravel()

# X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size=0.2, random_state=42)

# X_new1 = np.array(X_new1)
# y_new1 = np.array(y_new1).ravel()


(2872, 960)
(2872, 1)
(342, 3380)
(342, 1)


### 寻找最佳参数

In [3]:
import statistics
from sklearn.metrics import balanced_accuracy_score

# dataset splitting
from sklearn.model_selection import train_test_split
X_train_whole, X_ind_test, y_train_whole, y_ind_test =  train_test_split( X_new, y_new, test_size=0.2, random_state=1111)

print(X_train_whole.shape)
print(X_ind_test.shape)

(2297, 400)
(575, 400)


In [ ]:
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import confusion_matrix, roc_auc_score
import numpy as np
import pandas as pd
from xgboost import XGBClassifier  # 导入XGBoost分类器

# 假设 X_new 和 y_new 是你的数据集
X_train_whole, X_ind_test, y_train_whole, y_ind_test = train_test_split(X_new, y_new, test_size=0.2, random_state=1111)

BACC_collection = []
ACC_collection = []
Sn_collection = []
Sp_collection = []
MCC_collection = []
AUC_collection = []

# Initialize lists to store predictions and true values for each fold
y_true_all = []
y_pred_all = []
y_pred_proba_all = []

mean_recall = np.linspace(0, 1, 100)
all_precision = []
base_fpr = np.linspace(0, 1, 100)
mean_tpr = 0.0
interp_tpr_collection = []

def categorical_probas_to_classes(p):
    return np.argmax(p, axis=1)

# 初始化XGBoost分类器
clf = XGBClassifier(
    learning_rate=0.1,  # 学习率
    n_estimators=100,  # 树的个数
    max_depth=5,  # 树的深度
    random_state=1111  # 随机数的种子
)

skf = StratifiedKFold(n_splits=10)
for train, test in skf.split(X_train_whole, y_train_whole):
    X_train, X_valid, y_train, y_valid = X_train_whole[train], X_train_whole[test], y_train_whole[train], y_train_whole[test]
    clf.fit(X_train, y_train)

    y_pred_proba = clf.predict_proba(X_valid)
    y_valid_pred = categorical_probas_to_classes(y_pred_proba)

    y_true_all.extend(y_valid.ravel().astype(int))
    y_pred_all.extend(y_valid_pred.astype(int))
    y_pred_proba_all.extend(y_pred_proba[:, 1])

    TP, FP, FN, TN = confusion_matrix(y_valid, y_valid_pred).ravel()
    Sn_collection.append(TP / (TP + FN))
    Sp_collection.append(TN / (TN + FP))
    MCC = (TP * TN - FP * FN) / np.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))
    MCC_collection.append(MCC)
    ACC_collection.append((TP + TN) / (TP + TN + FP + FN))
    BACC_collection.append(0.5 * (TP / (TP + FN)) + 0.5 * (TN / (TN + FP)))
    auc = roc_auc_score(y_valid, y_pred_proba[:, 1])
    AUC_collection.append(auc)

results_df = pd.DataFrame({
    'Predicted_Proba': np.round(y_pred_proba_all, 8),
    'True_Label': y_true_all,
    'Predicted_Label': y_pred_all
})

# Save the results to a CSV file
results_df.to_csv(r'E:/Experiment\work/result/10折/XGB单个特征/XGB_DDE.csv', index=False)

# 输出结果
print(round(np.mean(ACC_collection), 3), '±', round(np.std(ACC_collection), 3))
print(round(np.mean(Sn_collection), 3), '±', round(np.std(Sn_collection), 3))
print(round(np.mean(Sp_collection), 3), '±', round(np.std(Sp_collection), 3))
print(round(np.mean(MCC_collection), 3), '±', round(np.std(MCC_collection), 3))
print(round(np.mean(AUC_collection), 3), '±', round(np.std(AUC_collection), 3))

In [5]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import balanced_accuracy_score

# Define the parameter grid for XGB
param_grid = {
    'n_estimators': (100,1000,100),  # 缩小范围，选择更合理的值
    'learning_rate': (0.01, 0.1,0.05),  # 学习率
    'max_depth': (1,20,2) # 树的最大深度
    # 'subsample': (0.6, 1.0,0.1),  # 子采样率
    # 'colsample_bytree': (0.6, 1.0,0.1)  # 列采样率
}

# Create an XGB classifier
clf = XGBClassifier()

# Create stratified k-fold cross-validation
skf = StratifiedKFold(n_splits=5)

# Perform grid search with cross-validation
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=skf, scoring='roc_auc')
grid_search.fit(X_new, y_new)

# Get the best parameters and corresponding mean cross-validation score
best_params = grid_search.best_params_
mean_score = grid_search.best_score_

# Get cross-validation scores for all parameter combinations
cv_results = grid_search.cv_results_

# Print the best parameters and mean cross-validation score
print("Best Parameters:", best_params)
print("Mean Cross-Validation Balanced Accuracy:", round(mean_score, 3))


Best Parameters: {'learning_rate': 0.05, 'max_depth': 2, 'n_estimators': 1000}
Mean Cross-Validation Balanced Accuracy: 0.712


In [70]:
import math
import numpy as np
import statistics
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, roc_auc_score

# 初始化指标列表
AAC_collection = []
Sn_collection = []
Sp_collection = []
MCC_collection = []
AUC_collection = []

# 保存预测结果
y_true_all = []
y_pred_all = []
y_pred_proba_all = []

def categorical_probas_to_classes(p):
    return np.argmax(p, axis=1)



clf = XGBClassifier(
    n_estimators=503,
    max_depth=10,
    learning_rate=0.05, 
    # subsample=1,
    # colsample_bytree=1,
    scale_pos_weight=1627/1245,  # 计算好的类别不平衡比例
    random_state=42,
    n_jobs=-1
    # use_label_encoder=False,
    # eval_metric='auc'
)

# 五折交叉验证
skf = StratifiedKFold(n_splits=5)
for train_idx, test_idx in skf.split(X_new, y_new):
    X_train, X_valid = X_new[train_idx], X_new[test_idx]
    y_train, y_valid = y_new[train_idx], y_new[test_idx]

    clf.fit(X_train, y_train)
    y_pred_proba = clf.predict_proba(X_valid)
    y_valid_pred = categorical_probas_to_classes(y_pred_proba)

    # 保存预测结果
    y_true_all.extend(y_valid)
    y_pred_all.extend(y_valid_pred)
    y_pred_proba_all.extend(y_pred_proba[:, 1])  # 正类概率

    # 计算混淆矩阵及各项指标
    TN, FP, FN, TP = confusion_matrix(y_valid, y_valid_pred).ravel()
    Sn_collection.append(TP / (TP + FN))
    Sp_collection.append(TN / (TN + FP))
    AAC_collection.append((TP + TN) / (TP + TN + FP + FN))
    MCC = (TP * TN - FP * FN) / math.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))
    MCC_collection.append(MCC)
    auc = roc_auc_score(y_valid, y_pred_proba[:, 1])
    AUC_collection.append(auc)

# 保存结果为 CSV 文件
results_df = pd.DataFrame({
    'Predicted_Proba': np.round(y_pred_proba_all, 8),
    'True_Label': y_true_all,
    'Predicted_Label': y_pred_all
})
results_df.to_csv(r'E:/Experiment\work/result/10折/XGB单个特征/XGB_ESM Cambrian.csv', index=False)

# 输出平均值 ± 标准差
print("ACC:", round(statistics.mean(AAC_collection), 3), '±', round(statistics.stdev(AAC_collection), 3))
print("Sn :", round(statistics.mean(Sn_collection), 3), '±', round(statistics.stdev(Sn_collection), 3))
print("Sp :", round(statistics.mean(Sp_collection), 3), '±', round(statistics.stdev(Sp_collection), 3))
print("MCC:", round(statistics.mean(MCC_collection), 3), '±', round(statistics.stdev(MCC_collection), 3))
print("AUC:", round(statistics.mean(AUC_collection), 3), '±', round(statistics.stdev(AUC_collection), 3))


ACC: 0.648 ± 0.012
Sn : 0.492 ± 0.037
Sp : 0.768 ± 0.043
MCC: 0.272 ± 0.023
AUC: 0.692 ± 0.013


### 测试

In [55]:
import math
import joblib
import numpy as np
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, cross_val_predict, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, roc_auc_score, log_loss

# 拆分数据
# X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size=0.2, random_state=42)

# XGBoost 参数
xgb_params = {
    'n_estimators': 403,
    'max_depth': 10,
    'learning_rate': 0.05, 
    'subsample': 1,
    'colsample_bytree': 1,
    'scale_pos_weight': 1627 / 1245,  # 类别不平衡处理
    'random_state': 42,
    'n_jobs': -1,
    'use_label_encoder': False,
    'eval_metric':'auc'
}

# xgb_params = {
#     'n_estimators': 1500,             # 更多树，配合 early_stopping
#     'max_depth': 12,                  # 树更深，学习更复杂非线性
#     'learning_rate': 0.05,            # 更小步长，收敛更平缓
#     'subsample': 0.6,                 # 行采样 60%，增强泛化
#     'colsample_bytree': 0.6,          # 列采样 60%，防高维过拟合
#     'min_child_weight': 3,            # 限制叶子节点最小权重
#     'gamma': 0.5,                     # 分裂最小增益阈值
#     'reg_alpha': 2.0,                 # L1 正则化更强
#     'reg_lambda': 5.0,                # L2 正则化更强
#     'scale_pos_weight': 1627/1245,    # 处理类别不平衡
#     'objective': 'binary:logistic',
#     # 'eval_metric': ['auc','logloss'],
#     # 'use_label_encoder': False,
#     'tree_method': 'hist',            # faster, handles high‑dim well
#     'grow_policy': 'lossguide',       # 按损失导向生长，适合高维
#     'n_jobs': -1,
#     'random_state': 42
# }



# xgb_params = {
#     'n_estimators': 1000,            # 增加树的数量，配合 early_stopping
#     'max_depth': 10,                 # 略微加深，学习更复杂模式
#     'learning_rate': 0.02,           # 更小步长，配合更多迭代
#     'subsample': 0.7,                # 行采样比例，防过拟合
#     'colsample_bytree': 0.7,         # 列采样比例，防过拟合
#     'min_child_weight': 5,           # 叶子节点最小权重和
#     'gamma': 0.3,                    # 分裂增益阈值
#     'reg_alpha': 1.0,                # L1 正则化
#     'reg_lambda': 3.0,               # L2 正则化
#     'scale_pos_weight': 1627/1245,   # 处理类别不平衡
#     'objective': 'binary:logistic',  
#     'eval_metric': ['auc','logloss'],# 同时监控 AUC 和 logloss
#     'use_label_encoder': False,
#     'n_jobs': -1,
#     'random_state': 42
# }



# 创建包含标准化的管道
model = make_pipeline(
    StandardScaler(),
    xgb.XGBClassifier(**xgb_params)
)

# 5折交叉验证预测
cv = StratifiedKFold(n_splits=5)
y_pred = cross_val_predict(model, X_new, y_new, cv=cv, method='predict')
y_proba = cross_val_predict(model, X_new, y_new, cv=cv, method='predict_proba')[:, 1]

# 模型拟合（用于提取特征重要性）
model.fit(X_new, y_new)

# 安全除法函数
def safe_divide(a, b):
    return a / b if b != 0 else 0

# 混淆矩阵和指标
cm = confusion_matrix(y_new, y_pred)
TN, FP, FN, TP = cm.ravel()

AAC = (TP + TN) / (TP + TN + FP + FN)
Sn = safe_divide(TP, TP + FN)
Sp = safe_divide(TN, TN + FP)
MCC_num = (TP * TN - FP * FN)
MCC_den = math.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))
MCC = safe_divide(MCC_num, MCC_den)
AUC = roc_auc_score(y_new, y_proba)
logloss = log_loss(y_new, y_proba)

# 打印结果
print("\n========== 交叉验证评估报告（XGBoost） ==========")
print(f"准确率 (AAC): {AAC:.4f}")
print(f"灵敏度 (Sn): {Sn:.4f}")
print(f"特异度 (Sp): {Sp:.4f}")
print(f"马修斯系数 (MCC): {MCC:.4f}")
print(f"AUC分数: {AUC:.4f}")
print(f"对数损失: {logloss:.4f}")
print("混淆矩阵:")
print(cm)

# 特征重要性
xgb_model = model.named_steps['xgbclassifier']
importances = xgb_model.feature_importances_

print("\nTop 10重要特征:")
sorted_idx = importances.argsort()[::-1]
feature_names = X_new.columns if hasattr(X_new, 'columns') else [f'Feature_{i}' for i in range(X_new.shape[1])]
for i in sorted_idx[:10]:
    print(f"{feature_names[i]}: {importances[i]:.4f}")



========== 交叉验证评估报告（XGBoost） ==========
准确率 (AAC): 0.5575
灵敏度 (Sn): 0.4651
特异度 (Sp): 0.6281
马修斯系数 (MCC): 0.0938
AUC分数: 0.5606
对数损失: 0.7772
混淆矩阵:
[[1022  605]
 [ 666  579]]

Top 10重要特征:
Feature_239: 0.0078
Feature_60: 0.0078
Feature_179: 0.0073
Feature_234: 0.0067
Feature_283: 0.0067
Feature_161: 0.0066
Feature_171: 0.0060
Feature_124: 0.0058
Feature_235: 0.0057
Feature_255: 0.0056


In [ ]:

# 独立测试
import math
import numpy as np
import statistics
# import matplotlib.pyplot as plt
# from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix,roc_auc_score

# BACC_collection = []
AAC_collection = []
Sn_collection = []
Sp_collection = []
MCC_collection = []
AUC_collection = []


def categorical_probas_to_classes(p):
    return np.argmax(p, axis=1)

for i in range(2):
    # dataset splitting
    # X_train_whole, X_ind_test, y_train_whole, y_ind_test = train_test_split(X_new, y_new, test_size=0.2, random_state=1111)
    
    # Create and train Decision Tree classifier
    # clf = RandomForestClassifier(max_depth=600, max_features=1, n_estimators=1000, n_jobs=-1)  # need to tune the hyperparameters
    model.fit(X_new, y_new)
    # clf = joblib.load('mode\RF_ESM300/rf_ESM300_model_fold_5.joblib')
    # Predict on independent test set
    y_pred_score = model.predict_proba(X_new1)
    y_pred = categorical_probas_to_classes(y_pred_score)
    y_true = y_new1
    TP, FP, FN, TN = confusion_matrix(y_true, y_pred).ravel()
    Sn_collection.append(TP / (TP + FN))
    AAC_collection.append((TP + TN) / (TP + TN + FP + FN))
    Sp_collection.append(TN / (TN + FP))
    MCC = (TP * TN - FP * FN) / math.pow(((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN)), 0.5)
    MCC_collection.append(MCC)
    auc = roc_auc_score(y_true, y_pred_score[:, 1])
    AUC_collection.append(auc)
  
# Print results
print(round(statistics.mean(AAC_collection), 3), '±', round(statistics.stdev(AAC_collection), 3))
print(round(statistics.mean(Sn_collection), 3), '±', round(statistics.stdev(Sn_collection), 3))
print(round(statistics.mean(Sp_collection), 3), '±', round(statistics.stdev(Sp_collection), 3))
print(round(statistics.mean(MCC_collection), 3), '±', round(statistics.stdev(MCC_collection), 3))
print(round(statistics.mean(AUC_collection), 3), '±', round(statistics.stdev(AUC_collection), 3))
